In [ ]:
!pip install transformers
!pip install sentense-piece

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torch.nn as nn
import torch.nn.functional as F
import os
device='cuda' if torch.cuda.is_available() else 'cpu'
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
trans=transforms.Compose([
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.GaussianBlur(kernel_size=3, sigma=0.1),
    torchvision.transforms.ColorJitter(0.2,0.2,0.2)
])

In [ ]:
from transformers import AutoFeatureExtractor
class DataSet2(Dataset):
    def __init__(self,paths,labels,transform=None):
        self.transform=transform
        self.images_path=paths
        self.labels=labels
        self.transform_norm=transforms.Normalize((0.485, 0.456, 0.406), 
                                              (0.229, 0.224, 0.225))
        if(transform is not None):
            self.images_path=np.append(self.images_path,self.images_path)
            self.labels=np.append(self.labels,self.labels)
    def __len__(self):
        return len(self.images_path)
    def __getitem__(self,idx):
        hf=len(self.images_path)/2
        img=self.images_path[idx]
        image=cv2.imread(img)
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image,(224,224), interpolation=cv2.INTER_CUBIC)
        resized_image=AutoFeatureExtractor.from_pretrained("martinezomg/vit-base-patch16-224-diabetic-retinopathy")(resized_image).pixel_values
        resized_image=np.array(resized_image)
        resized_image=torch.tensor(resized_image).to(torch.float32)
        if(self.transform is not None):
            half=len(self.images_path)/2
            if(idx>=half):
                resized_image=torch.reshape(resized_image,(3,224,224))
                resized_image=self.transform(resized_image)
                return resized_image,self.labels[idx]
            
        resized_image=torch.reshape(resized_image,(3,224,224))
        resized_image=self.transform_norm(resized_image)
        return resized_image,self.labels[idx]

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,f1_score
def get_acc(pred,out):
    arr=pred.to('cpu').detach()
    arr=np.array(arr)
    arr2=out.to('cpu').detach()
    arr2=np.array(arr2)
    return accuracy_score(arr,arr2)
def get_prec(pred,out):
    arr=pred.to('cpu').detach()
    arr=np.array(arr)
    arr2=out.to('cpu').detach()
    arr2=np.array(arr2)
    return precision_score(arr,arr2,average='macro')
def get_f1(pred,out):
    arr=pred.to('cpu').detach()
    arr=np.array(arr)
    arr2=out.to('cpu').detach()
    arr2=np.array(arr2)
    return f1_score(arr,arr2,average='macro')

In [ ]:
import pickle
def save_model(list_models,filename):
    pickle.dump(list_models, open(filename, 'wb'))
def load_model(filename):
    list_models=pickle.load(open(filename, 'rb'))
    return list_models
filename='/content/drive/MyDrive/DL_project/Models/Attention'

In [ ]:
import pickle
def save_model(list_models,filename):
    pickle.dump(list_models, open(filename, 'wb'))
def load_model(filename):
    list_models=pickle.load(open(filename, 'rb'))
    return list_models
filename='/content/drive/MyDrive/DL_project/Models/Attention'

In [ ]:
from sklearn.model_selection import train_test_split
train_data=os.listdir("/content/drive/MyDrive/DL_project/TEST")
train_data.sort()
train_data_2=[0,0,0,0,0]
train_data_2[0]=train_data[1]
train_data_2[1]=train_data[2]
train_data_2[2]=train_data[0]
train_data_2[3]=train_data[4]
train_data_2[4]=train_data[3]
all_paths=[]
all_labels=[]
for i,path in enumerate(train_data_2):
    full_path="/content/drive/MyDrive/DL_project/TEST/"+path+"/"
    lst=os.listdir(full_path)
    lst=[full_path+i for i in lst]
    all_paths.extend(lst)
    all_labels.extend([i for j in range(len(lst))])
all_paths=np.array(all_paths)
all_labels=np.array(all_labels)

In [ ]:
test_dataset=DataSet2(all_paths,all_labels,trans)
test_loader=DataLoader(test_dataset,batch_size=12,shuffle=True)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
model.eval()
with torch.no_grad():
    total_loss = 0
    test_acc=0
    test_prec=0
    test_f1=0
    testing_pred=None
    testing_out=None
    for i,(inputs, targets) in tqdm(enumerate(test_loader)):
        outputs = model(inputs)
        targets=targets.to(torch.long)
        targets=targets.to('cpu')
        outputs=outputs.to('cpu')
        loss = criterion(outputs, targets)
        total_loss += loss.item()
        outputs=torch.argmax(outputs,dim=1)
        test_acc+=get_acc(outputs,targets)
        test_prec+=get_prec(outputs,targets)
        test_f1+=get_f1(outputs,targets)
        if(testing_pred is None):
            testing_pred=np.array(outputs)
            testing_out=np.array(targets)
        else:
            testing_pred=np.concatenate([testing_pred,np.array(outputs)],axis=0)
            testing_out=np.concatenate([testing_out,np.array(targets)],axis=0)        
    test_acc/=(i+1)
    total_loss/=(i+1)
    test_prec/=(i+1)
    test_f1/=(i+1)
    sen = f"Testing ,Running Loss {total_loss} ,Accuracy:{test_acc},Precision:{test_prec},F1:{test_f1}"
    print(sen)
    cm = confusion_matrix(testing_out,testing_pred, labels=[i for i in range(5)])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[i for i in range(5)])
    disp.plot()
    plt.show()